In [1]:
import open3d as o3d
import numpy as np
import copy

import time

In [2]:
# transforem the source cloud and then draw the point clouds
def draw_registration_result(source, target, transformation):
    #copy original point clouds
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #set the color
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    #apply transformation
    source_temp.transform(transformation)
    #draw the clouds
    o3d.visualization.draw_geometries([source])
    o3d.visualization.draw_geometries([target])

In [3]:
#down sample the point cloud, recompute new normals, compute fpfh features
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    #use the neighbors recompute the normal
    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    #compute the FPFH features
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh



In [4]:
#load data, down sample and compute fpfh feature
def prepare_dataset(voxel_size):
    #load point clouds
    print(":: Load two point clouds and disturb initial pose.")
    source = o3d.io.read_point_cloud("badslam_scan0422/good3_bg.ply")
    target = o3d.io.read_point_cloud("badslam_scan0422/ball/ball.ply")
    
    #apply an inital transformation
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    #visualize
    draw_registration_result(source, target, np.identity(4))

    #downsample and compute fpfh feature of the point clouds
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [5]:
#global registration based on RANSAC
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    
    #apply RANSAC algorithms for registration:
    #pick 4 points,
    #pruning with edge length and distance for early stop
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

In [6]:
#fast global registration
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [7]:
#point to plane registration
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.registration.registration_icp(
        source, target, distance_threshold, result_fast.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    return result

In [8]:
voxel_size = 0.05  # means 5cm for the dataset

#down sample, compute fpfh of the data
source, target, source_down, target_down, source_fpfh, target_fpfh = \
        prepare_dataset(voxel_size)

#global registration using ransac
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print("Global registration took %.3f sec.\n" % (time.time() - start))
print(result_ransac)


#draw the result
#draw_registration_result(source_down, target_down,
#                         result_ransac.transformation)

#fast global registration
start = time.time()
result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
#draw_registration_result(source_down, target_down,
#                         result_fast.transformation)
    
#local refinement
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
Global registration took 0.654 sec.

registration::RegistrationResult with fitness=8.993279e-01, inlier_rmse=3.114507e-02, and correspondence_set size of 6021
Access transformation to get result.
:: Apply fast global registration with distance threshold 0.025
Fast global registration took 0.287 sec.

registration::RegistrationResult with fitness=4.800597e-01, inlier_rmse=1.815831e-02, and correspondence_set size of 3214
Access transformation to get result.
:: Point-to-plane ICP registration is applied on original point
   clouds t

In [9]:
#outlier->segment plane->cluster->find object with variance of normal and position
#select ouliers from icp algorithms
threshold = voxel_size*1

#get points with correspondence
cor_ind = np.asarray(result_icp.correspondence_set)
print(cor_ind)
target_out = target.select_down_sample(cor_ind[:,1],invert=True)
print(target_out)
o3d.visualization.draw_geometries([target_out])
##compute outlier distance
out_dist = np.asarray(target_out.compute_point_cloud_distance(source))

l_cor_s = np.shape(out_dist)
l = l_cor_s[0]
print(l_cor_s)

#select outlier index
out_index = np.arange(0,l,1)
out_list = out_index[out_dist>threshold]
target_out = target_out.select_down_sample(out_list)
o3d.visualization.draw_geometries([target_out])


#cluster
cluster_result = target_out.cluster_dbscan(voxel_size*0.35, 20, print_progress=False)
cluster_np = np.asarray(cluster_result)

clusters_points = []
clusters_points_visusal = []
clusters_ind = []
clusters_candidate = []
clusters_cand_points = []
clusters_scores = []
clusters_var_points = []
clusters_var_normals = []
clusters_distance = []
clusters_ratio = []
clusters_number = []
t=0
target_ind = np.arange(0,np.size(cluster_np),1)
for cluster in range(max(cluster_np)+1):
    #get point cloud index of certain cluster
    cur_cluster_ind = target_ind[cluster_np==cluster]
    clusters_ind.append(cur_cluster_ind)
    #get point cloud of certain cluster
    cur_cluster_points = target_out.select_down_sample(cur_cluster_ind)
    clusters_points.append(cur_cluster_points)
    #visualization
    cur_cluster_temp = copy.deepcopy(cur_cluster_points)
    temp_value = np.random.rand(1,3)
    cur_cluster_temp.paint_uniform_color([temp_value[0][0], temp_value[0][1], temp_value[0][2]])
    clusters_points_visusal.append(cur_cluster_temp)

o3d.visualization.draw_geometries(clusters_points_visusal)
flat_clusters_ind = np.concatenate(clusters_ind,axis=0)
target_cluster = target_out.select_down_sample(flat_clusters_ind)

[[ 59423 260484]
 [ 59424 198584]
 [ 59425 104463]
 ...
 [ 59419  73594]
 [ 59420 292816]
 [ 59421 321828]]
geometry::PointCloud with 254180 points.
(254180,)


In [10]:
##segment out planes in each cluster
o3d.visualization.draw_geometries([clusters_points[6]])
num_cls = np.max(cluster_np)+1
print(num_cls)
clusters_plane = []
for n_cls in range(num_cls):
    #get size of the cluster
#     print(n_cls)
    num_points = np.size(clusters_ind[n_cls])
    #segment a plane in the cluster
    plane,points_on_plane = clusters_points[n_cls].segment_plane(0.007,4,1000)
#     print("segment_plane")
    #get index of plane points
    inds_plane = clusters_ind[n_cls][points_on_plane]
#     print("inds_plnae")
    #jurge whether the plane is from the object or from the desk.
    #1. get ind without plane
    inds_not_on_plane = np.setdiff1d(clusters_ind[n_cls],inds_plane)
    #2. get points and points without plane.
    post = target_out.select_down_sample(inds_not_on_plane)
    pre_points = np.asarray(clusters_points[n_cls].points)
    post_points = np.asarray(post.points)
    #3. calculate varience of normalized directions
    pre_mean_point=np.mean(pre_points,axis = 0)
    #print(pre_mean_point)
    post_mean_point = np.mean(post_points,axis = 0)
    pre_drct = (pre_points-pre_mean_point)/np.linalg.norm(pre_points-pre_mean_point,axis=1)[:,None]
    post_drct = (post_points-post_mean_point)/np.linalg.norm(post_points-post_mean_point,axis=1)[:,None]
    pre_var = np.sum(np.var(pre_drct))
    post_var = np.sum(np.var(post_drct))
#     print("post_pre")
    #4. judge and record: after segmentation become two objects, or itself is a plane
    cluster_n=0
    
    if not np.size(post_points,0)==0:
#         print(np.size(post_points,0))
        post_cluster = post.cluster_dbscan(voxel_size*0.5, 10, print_progress=False)
        #get cluster
        post_cluster = np.asarray(post_cluster)
        if not np.size(post_cluster)==0:        
#             print(post_cluster)
            cluster_n = max(post_cluster)
#             print("cluster_n")
                
    #print(cluster_n)
    if pre_var<post_var or np.size(inds_plane)>num_points*0.8 or cluster_n>0:
        clusters_plane.append(inds_plane)



162


c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\numpy\core\_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\numpy\core\fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

In [11]:
#plane points ind
flat_clusters_plane = np.concatenate(clusters_plane,axis=0)

#get remaining points index
flat_cls_ind_no_plane = np.setdiff1d(flat_clusters_ind,flat_clusters_plane)

target_plane = target_out.select_down_sample(flat_clusters_plane)
target_cluster_no_plane = target_out.select_down_sample(flat_cls_ind_no_plane)

#visualization
o3d.visualization.draw_geometries([target_plane])
o3d.visualization.draw_geometries([target_cluster_no_plane])

In [12]:
#cluster the point cloud:
cluster_new = np.asarray(target_cluster_no_plane.cluster_dbscan(voxel_size*0.15, 10, print_progress=False))
print(cluster_new)
num_cls_new = max(cluster_new)+1

clusters_points_new = []
clusters_points_visusal_new = []
clusters_ind_new = []
for cluster in range(max(cluster_new)+1):
    #get point cloud index of certain cluster
    cur_cluster_ind_new = flat_cls_ind_no_plane[cluster_new==cluster]
    clusters_ind_new.append(cur_cluster_ind_new)
    #get point cloud of certain cluster
    cur_cluster_points_new = target_out.select_down_sample(cur_cluster_ind_new)
    clusters_points_new.append(cur_cluster_points_new)
    cur_cluster_temp = copy.deepcopy(cur_cluster_points_new)
    temp_value = np.random.rand(1,3)
    cur_cluster_temp.paint_uniform_color([temp_value[0][0], temp_value[0][1], temp_value[0][2]])
    clusters_points_visusal_new.append(cur_cluster_temp)

#visualization
#o3d.visualization.draw_geometries(clusters_points)
flat_clusters_ind_new = np.concatenate(clusters_ind_new,axis=0)
target_cluster_new = target_out.select_down_sample(flat_clusters_ind_new)
o3d.visualization.draw_geometries([target_cluster_new])
o3d.visualization.draw_geometries(clusters_points_visusal_new)

[ 0  0  1 ... -1 -1 -1]


In [13]:
#find the cluster with most variance of normals and least variance of position
mean_coordinate = np.mean(np.asarray(target_out.points))
for cluster in range(max(cluster_new)+1):
    cur_cluster_ind = clusters_ind_new[cluster]

    #number of elements
    num_element = np.size(cur_cluster_ind)
    cur_cluster_points = clusters_points_new[cluster]
    #variance (defined) of points
    points = np.asarray(cur_cluster_points.points)
    mean_points=np.mean(points,axis=0)
    points_distance = np.linalg.norm(points-mean_points,axis=1)
    points_dist_mean = np.mean(points_distance)
    points_dist_var = np.var(points_distance)
    disperse = points_dist_var/points_dist_mean
    #variance of directions
    #print(mean_points.shape)
    points_drct = (points-mean_points)/np.linalg.norm(points-mean_points,axis=1)[:,None]
    #print(np.linalg.norm(points-mean_points,axis=1).shape)
    drct_var = np.sum(np.var(points_drct,axis=0))
    #print(np.var(points_drct,axis=0).shape)
    #variance of normal        
    normals = np.asarray(cur_cluster_points.normals)
    var_normals = np.sum(np.var(normals))
    #difference from center?
    distance = np.linalg.norm(mean_points-mean_coordinate)
        
    
    if num_element>1000 and num_element<40000 and var_normals>0.25:         
        if t==0:
            best_cluster = cluster
            best_score = -100
            t=1
            
        
        score = drct_var-disperse
        clusters_scores.append(score)
        if score>best_score:
            best_cluster = cluster
            best_score = score
        
        clusters_candidate.append(cluster)
        clusters_cand_points.append(target_out.select_down_sample(clusters_ind_new[cluster]))
        clusters_var_points.append(disperse)
        clusters_var_normals.append(var_normals)
        clusters_distance.append(distance)
        clusters_number.append(num_element)
        
print(clusters_candidate)
print(clusters_scores)
print(clusters_var_points)
print(clusters_var_normals)
print(clusters_distance)
print(clusters_number)
print(best_cluster)
print(clusters_ratio)       
        
max_cluster_ind = clusters_ind_new[best_cluster]
object_cloud = target_out.select_down_sample(max_cluster_ind)
o3d.visualization.draw_geometries(clusters_cand_points)
o3d.visualization.draw_geometries([object_cloud])

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


[0, 2, 4, 6, 30, 42]
[0.9904927695744757, 0.9790210826254553, 0.9552921535298503, 0.9796021592118157, 0.9843441520772188, 0.9879471243560624]
[0.0030363915813062118, 0.015036026577830004, 0.023425409518284247, 0.011249714182243764, 0.014366890281823995, 0.011458718548097453]
[0.2919404149942897, 0.31304978170696174, 0.3152888237367161, 0.3330235562165256, 0.3253061142396513, 0.3239057151675743]
[0.2944956975131145, 0.12338283779399473, 0.35133597911977144, 0.47065555354115124, 0.4361301921732689, 0.5429392033668641]
[23479, 12594, 19467, 2670, 1199, 2341]
0
[]


In [15]:
o3d.io.write_point_cloud("objects/ball_object.pcd",object_cloud)

True